In [136]:
from IPython.display import display
import numpy as np
import pandas as pd
import nltk
import re
import string
import random
from random import shuffle
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.classify import NaiveBayesClassifier
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI

#nltk.download('stopwords')
random.seed(10)

STOPWORDS = set(nltk.corpus.stopwords.words('english'))
REGEX = re.compile('[%s]' % re.escape(string.punctuation))
STEMMER = SnowballStemmer('english')
LEMMATIZER = WordNetLemmatizer()

# Text Classification (Naive Bayes)

In [137]:
movie_reviews = pd.read_csv('../movie_data.csv')
display(movie_reviews.head())

,movie,reviewer,metascore,review_score,review_text,review_date,sentiment,release_date,user_acclaim,user_score,...,Romance,SciFi,Short,Sport,TalkShow,Thriller,War,Western,unknown,release_decade
0,Notes on Blindness,Stephen Holden,75.0,90,The tone of the narration is so wrenchingly ho...,2016-11-15,pos,2016-11-16,No score yet,NaN,...,0,0,0,0,0,0,0,0,0,2010s
1,Devils on the Doorstep,Stephen Holden,70.0,80,"In its dry and forceful way, it delivers the s...",NaN,pos,2002-12-18,Universal acclaim,83.0,...,0,0,0,0,0,0,1,0,0,2000s
2,The Upside of Anger,Dana Stevens,63.0,60,A seriously flawed movie wrapped around two ne...,NaN,mixed,2005-03-11,Generally favorable reviews,80.0,...,1,0,0,0,0,0,0,0,0,2000s
3,Monster,Stephen Holden,74.0,70,The movie's biggest disappointment is the vagu...,NaN,pos,2003-12-24,Generally favorable reviews,67.0,...,1,0,0,0,0,0,0,0,0,2000s
4,Rock in the Red Zone,Ken Jaworowski,54.0,50,Rock in the Red Zone has its best moments when...,2015-11-12,mixed,2015-11-12,No score yet,NaN,...,0,0,0,0,0,0,1,0,0,2010s


# Etapa 0 - Funções

Para otimizar os resultados obtidos com os classificadores, é necessário fazer o tratamento dos textos das resenhas existentes no base de dados. Foram usadas tecnicas como remoção de stopwords, stemmização, tokenização por palavra (nltk word_tokenizer) e lemmatização. Para facilitar a leitura e entendimento do notebook, as funções de tratamento e categorização foram agrupadas em uma só celula, bem como as funções necessárias futuramente em classificadores especificos.

In [138]:
def word_parser(x):
    if x not in STOPWORDS:
        return ({STEMMER.stem(REGEX.sub('',x).lower()):True})


def create_word_features(text):
    words={}
    for word in word_tokenize(text):
        if (word not in STOPWORDS):
            words.update(word_parser(word))
    return words
    
def category_filter(category):
    reviews = []
    for x in movie_reviews.review_text[(movie_reviews.sentiment == category)]:   
        reviews.append((create_word_features(x),category))
    for x in reviews:
        if '' in x[0]:
            del x[0]['']
    return reviews

def predict(model,text):
    words = {}
    for x in word_tokenize(text):
        parsed_word = word_parser(LEMMATIZER.lemmatize(x))
        if parsed_word:
            words.update(parsed_word)
    return model.classify(words)

def mode(lista):
    counts = [lista.count(x) for x in lista]
    return lista[counts.index(max(counts))]
    

# Etapa 1 - Features

Após aplicar as tecnicas de tratamento às palavras utilizando as funções acima, é necessário categorizar e criar features para cada classe de palavra. Os classificadores serão treinados com 3 classes: 'pos', 'neg' e 'mixed', que representam, respectivamente, os sentimentos 'positivo', 'negativo' e 'neutro'. Aplicaremos então uma função de aleatorização à lista de todas as palavras do banco de dados já categorizadas para prosseguir com a criação do conjunto de treinamento (70% do total de reviews) e de teste (30% do total de reviews).

In [139]:
pos = category_filter('pos')
neg = category_filter('neg')
mixed = category_filter('mixed')

all_classes = pos+neg+mixed
shuffle(all_classes)

print('Positive reviews:',len(pos)) 
print('Mixed reviews:',len(mixed))
print('Negative reviews:',len(neg))

Positive reviews: 5621
Mixed reviews: 5147
Negative reviews: 1642


In [140]:
train_set = all_classes[:8680]
test_set = all_classes[8680:]

print('Train set: {} reviews'.format(len(train_set)))
print('Test set: {} reviews'.format(len(test_set)))


Train set: 8680 reviews
Test set: 3730 reviews


# Etapa 2 - Classificadores

Nesta etapa, usando nltk e sua integração com sklearn, treinaremos diversos classificadores com o conjunto de treinamento definido prosteriormente. 

### Naive Bayes Classifier

In [141]:
NB_classifier = NaiveBayesClassifier.train(train_set)

### Multinomial Naive Bayes Classifier

In [142]:
MNB_classifier = SklearnClassifier(MultinomialNB()).train(train_set)

### Logistic Regression Classifier

In [143]:
LR_classifier = SklearnClassifier(LogisticRegression()).train(train_set)

### SGD Classifier

In [144]:
SGD_classifier = SklearnClassifier(SGDClassifier()).train(train_set)

### SVC Classifier (quebrado)

In [145]:
SVC_classifier = SklearnClassifier(SVC()).train(train_set)

### Linear SVC Classifier

In [146]:
LinearSVC_classifier = SklearnClassifier(LinearSVC()).train(train_set)

### NuSVC Classifier

In [147]:
NuSVC_classifier = SklearnClassifier(NuSVC(nu=0.3)).train(train_set)

### Classificador por votos

Agora que treinamos diversos classificadores disponíveis no sklearn e nltk, criaremos nosso próprio classficador. Ele funcionará como uma espécie de conselho, onde coletará o resultado de 6 outros classificadores e, baseado neles, retornará o resultado com mais ocorrencia entre eles. 

In [148]:
class CustomClassifier(ClassifierI):
    
    def __init__(self,*classifiers):
        self._classifiers = classifiers
    
    def classify(self, feature_set):
        return mode([classifier.classify(feature_set) for classifier in self._classifiers]) 
    
    def prob(self, feature_set):
        votes = [classifier.classify(feature_set) for classifier in self._classifiers]
        return votes.count(mode(votes)) / len(votes)

C_classifier = CustomClassifier(NB_classifier, 
                                     MNB_classifier,
                                     LR_classifier,
                                     SGD_classifier,
                                     SVC_classifier,
                                     LinearSVC_classifier,
                                     NuSVC_classifier)

In [149]:
#função para uso futuros
def trainClassifiers(train_set):
    NB_classifier = NaiveBayesClassifier.train(train_set)
    LR_classifier = SklearnClassifier(LogisticRegression()).train(train_set)
    SGD_classifier = SklearnClassifier(SGDClassifier()).train(train_set)
    SVC_classifier = SklearnClassifier(SVC()).train(train_set)
    LinearSVC_classifier = SklearnClassifier(LinearSVC()).train(train_set)
    NuSVC_classifier = SklearnClassifier(NuSVC(nu=0.3)).train(train_set)
    NB_classifier = NaiveBayesClassifier.train(train_set)
    C_classifier = CustomClassifier(NB_classifier, 
                                     MNB_classifier,
                                     LR_classifier,
                                     SGD_classifier,
                                     SVC_classifier,
                                     LinearSVC_classifier,
                                     NuSVC_classifier)

# Etapa 3 - Acurácia

Criados e treinados os classificadores, verificaremos a acurácia de cada um deles usando o nosso conjunto de teste definido na etapa 1 deste notebook.

In [150]:
def testAccuracy(test_set):
    print("\nNB Accuracy: %.2f" %(nltk.classify.util.accuracy(NB_classifier, test_set)*100) + "%")
    print("\nMNB Accuracy: %.2f" %(nltk.classify.util.accuracy(MNB_classifier, test_set)*100) + "%")
    print("\nLR Accuracy: %.2f" %(nltk.classify.util.accuracy(LR_classifier, test_set)*100) + "%")
    print("\nSGD Accuracy: %.2f" %(nltk.classify.util.accuracy(SGD_classifier, test_set)*100) + "%")
    print("\nSVC Accuracy: %.2f" %(nltk.classify.util.accuracy(SVC_classifier, test_set)*100) + "%")
    print("\nLinear SVC Accuracy: %.2f" %(nltk.classify.util.accuracy(LinearSVC_classifier, test_set)*100) + "%")
    print("\nNuSVC Accuracy: %.2f" %(nltk.classify.util.accuracy(NuSVC_classifier, test_set)*100) + "%")
    print("\nC_Classifier Accuracy: %.2f" %(nltk.classify.util.accuracy(C_classifier,test_set)*100) + "%")

testAccuracy(test_set)


NB Accuracy: 57.56%

MNB Accuracy: 60.13%

LR Accuracy: 59.12%

SGD Accuracy: 55.98%

SVC Accuracy: 43.91%

Linear SVC Accuracy: 54.96%

NuSVC Accuracy: 55.50%

C_Classifier Accuracy: 58.28%


Verificamos, após diversos testes com esses classificadores, que o Multinomial Naive Bayes e o nosso classificador por votos se destacam ao mostrar, em quase todos, as maiores acurácias para a nossa base de dados. No entanto, o SVC apresentou uma acurácia muito abaixo da média.

# Etapa 4 - Investigando bias

Para investigar se existe alguma polarização no julgamento dos classificadores, nesta etapa testaremos a acurácia de cada um deles para apenas reviews negativos e para apenas reviews positivos. 

In [151]:
#Negative

pos = category_filter('pos')
neg = category_filter('neg')
mixed = category_filter('mixed')

all_classes = pos+neg[:1000]+mixed
shuffle(all_classes)

print('Positive reviews: {}'.format(len(pos))) 
print('Mixed reviews: {}'.format(len(mixed)))
print('Negative reviews: {}\n'.format(len(neg)))

train_set = all_classes
test_set = neg[1000:]

print('Train set: {} reviews'.format(len(train_set)))
print('Test set: {} reviews'.format(len(test_set)))

trainClassifiers(train_set)
testAccuracy(test_set)

Positive reviews: 5621
Mixed reviews: 5147
Negative reviews: 1642

Train set: 11768 reviews
Test set: 642 reviews

NB Accuracy: 72.12%

MNB Accuracy: 37.38%

LR Accuracy: 64.17%

SGD Accuracy: 64.49%

SVC Accuracy: 0.00%

Linear SVC Accuracy: 73.68%

NuSVC Accuracy: 65.11%

C_Classifier Accuracy: 65.42%


Verificamos, após a primeira investigação, uma acurácia de 0% no classificador SVC, resultado extremamente improvável e que indica bias. Além disso, ao testarmos várias vezes, temos um resultado bastante heterogêneo dos outros classificadores, que pode ser causado pela baixa quantidade de amostras negativas no conjunto de treinamento, que tem apenas 1000 reviews negativos em comparação com os mais de 10000 reviews positivos e mixed. Para confirmar o bias do classificador SVC, testaremos os modelos com um conjunto de teste unicamente positivo.  

In [152]:
#Positive

pos = category_filter('pos')
neg = category_filter('neg')
mixed = category_filter('mixed')

all_classes = pos[1000:]+neg+mixed
shuffle(all_classes)

print('Positive reviews: {}'.format(len(pos))) 
print('Mixed reviews: {}'.format(len(mixed)))
print('Negative reviews: {}\n'.format(len(neg)))

train_set = all_classes
test_set = pos[:1000]

print('Train set: {} reviews'.format(len(train_set)))
print('Test set: {} reviews'.format(len(test_set)))

trainClassifiers(train_set)
testAccuracy(test_set)

Positive reviews: 5621
Mixed reviews: 5147
Negative reviews: 1642

Train set: 11410 reviews
Test set: 1000 reviews

NB Accuracy: 84.50%

MNB Accuracy: 87.50%

LR Accuracy: 90.10%

SGD Accuracy: 87.50%

SVC Accuracy: 100.00%

Linear SVC Accuracy: 90.40%

NuSVC Accuracy: 83.90%

C_Classifier Accuracy: 92.20%


Verificamos, após testar os modelos com amostras positivas, o SVC apresenta uma acurácia de 100%, o que confirma a hipótese de que ele estava polarizado. Vemos também que os outros classificadores tiveram maior facilidade em detectar reviews positivos em comparação com os negativos. Essa diferença pode ser causada por um conjunto de dados desequilibrado. Seguimos, então, com o teste para a categoria 'mixed'.

In [153]:
#Mixed

pos = category_filter('pos')
neg = category_filter('neg')
mixed = category_filter('mixed')

all_classes = pos+neg+mixed[1000:]
shuffle(all_classes)

print('Positive reviews: {}'.format(len(pos))) 
print('Mixed reviews: {}'.format(len(mixed)))
print('Negative reviews: {}\n'.format(len(neg)))

train_set = all_classes
test_set = mixed[:1000]

print('Train set: {} reviews'.format(len(train_set)))
print('Test set: {} reviews'.format(len(test_set)))

trainClassifiers(train_set)
testAccuracy(test_set)

Positive reviews: 5621
Mixed reviews: 5147
Negative reviews: 1642

Train set: 11410 reviews
Test set: 1000 reviews

NB Accuracy: 73.40%

MNB Accuracy: 79.90%

LR Accuracy: 83.10%

SGD Accuracy: 78.30%

SVC Accuracy: 0.00%

Linear SVC Accuracy: 83.60%

NuSVC Accuracy: 74.80%

C_Classifier Accuracy: 79.60%


Os resultados se aproximam bastante dos resultados do teste para reviews positivos, o que é esperado considerando a quantidade parecida de reviews para as duas classes. O SVC, assim como no teste com reviews negativos, também mostra uma acurácia de 0%, reforçando a confirmação de que o classificador está classificando qualquer review como positivo e por isso será descartado.

# Etapa 4 - Predições

Para finalizar, faremos algumas predições e veremos o resultado de cada classificador para cada uma delas. É possível ver também que o classificador baseado em votos feito por nós está, de fato, retornando a resposta mais comum entre os outros classificadores.

In [166]:
def classifiers_predict(review):
    print('\nPredições para: {}\n-----------'.format(review))

    print("NB Classifier: {}\n".format(predict(NB_classifier, review)))
    print("MNB Classifier: {}\n".format(predict(MNB_classifier, review)))
    print("LRClassifier: {}\n".format(predict(LR_classifier, review)))
    print("SGD Classifier: {}\n".format(predict(SGD_classifier, review)))
    print("Linear SVC Classifier: {}\n".format(predict(LinearSVC_classifier, review)))
    print("NuSVC Classifier: {}\n".format(predict(NuSVC_classifier, review)))
    print("Custom Classifier: {}\n".format(predict(C_classifier, review)))
    
classifiers_predict("What a terrible movie, I was bored the whole time!")


Predições para: What a terrible movie, I was bored the whole time!
-----------
NB Classifier: neg

MNB Classifier: mixed

LRClassifier: neg

SGD Classifier: neg

Linear SVC Classifier: neg

NuSVC Classifier: neg

Custom Classifier: neg



In [167]:
classifiers_predict("I love this movie!")


Predições para: I love this movie!
-----------
NB Classifier: pos

MNB Classifier: pos

LRClassifier: pos

SGD Classifier: pos

Linear SVC Classifier: pos

NuSVC Classifier: pos

Custom Classifier: pos



In [168]:
classifiers_predict("Even though I think it's average, my wife loves it!")


Predições para: Even though I think it's average, my wife loves it!
-----------
NB Classifier: mixed

MNB Classifier: mixed

LRClassifier: pos

SGD Classifier: pos

Linear SVC Classifier: neg

NuSVC Classifier: pos

Custom Classifier: pos



# Etapa 5 - Outras Caracteristicas

A função "show_most_informative_features", do classificador Naive Bayes, é interessante pois mostra as palavras que são mais significativas para o modelo. Isto é, quando detectadas, aumentam consideravelmente a probabilidade do classificador acertar a classe do review testado. 

In [157]:
NB_classifier.show_most_informative_features(20)

Most Informative Features
                 tedious = True              neg : pos    =     34.4 : 1.0
                 incoher = True              neg : pos    =     20.6 : 1.0
                   lurch = True              neg : pos    =     20.2 : 1.0
                    flat = True              neg : pos    =     19.2 : 1.0
                  devast = True              pos : mixed  =     18.9 : 1.0
                daughter = True              neg : pos    =     17.8 : 1.0
                    poor = True              neg : pos    =     16.8 : 1.0
                portrait = True              pos : neg    =     15.6 : 1.0
                    sour = True              neg : pos    =     15.4 : 1.0
                  recycl = True              neg : pos    =     15.4 : 1.0
                    thud = True              neg : pos    =     15.4 : 1.0
                laughabl = True              neg : pos    =     15.4 : 1.0
                  dreari = True              neg : pos    =     14.9 : 1.0

Ao pedir ao classificador as 20 palavras mais informativas, é possível observar que reviews negativos carregam um conjunto de palavras mais especificas e comuns entre eles, apesar da pequena quantidade de amostras dessa classe no conjunto de dados. Tedious e Incoherent (incoher após a stemmização) aparecem, por exemplo, 34.4 e 20.6 vezes mais frequentemente em reviews negativos se comparados com positivos. 